### Read uuid file

In [ ]:
import os 
import csv
import yaml
uuids=[]
list_bbox=[]
with open('bbox_database.csv', 'rb') as mycsvfile:
    data = csv.reader(mycsvfile,delimiter=' ')
    for row in data:
        uuids.append(row[0])
        list_bbox.append(row[1])
        
       

In [ ]:
bbox_database=[]
for i in range(len(list_bbox)):
    
    temp=list_bbox[i][1:-1].split(',')
    x0=temp[0].replace("Decimal('",'').replace("')",'').lstrip().rstrip()
    y0=temp[1].replace("Decimal('",'').replace("')",'').lstrip().rstrip()
    x1=temp[2].replace("Decimal('",'').replace("')",'').lstrip().rstrip()
    y1=temp[3].replace("Decimal('",'').replace("')",'').lstrip().rstrip()
    bbox_database.append([x0,y0,x1,y1])




In [ ]:
#example
print bbox_database[4545]

In [ ]:
subset=uuids[:10]
#0 if is a valid mapproxy config - it returns a yaml, 1 if its invalid
valid_config= [None for i in range (len(subset))]
#0 if it is between -180,-90, 180, 90, 1 if it is outside that range or is None
valid_bbox = [None for i in range (len(subset))]
#0 is returns image, 1 if not
imag_exist= [1 for i in range (len(subset))]


### Generate yaml from uuid list

In [ ]:
import requests
import yaml

from multiprocessing import Pool

piensa_url = 'http://hhypermap.wall.piensa.co/registry/hypermap/layer'
yaml_folder = 'yamls'

if not os.path.isdir(yaml_folder):
    os.mkdir(yaml_folder)

def create_yaml_file(uuid):
    response = requests.get('{0}/{1}/map/config'.format(piensa_url, uuid))
    raw_content = response.content
    
    if 'h1 { font-weight:normal; }' in raw_content:
        return

    with open(os.path.join(yaml_folder, '{0}.yml'.format(uuid)), 'wb') as f:
        f.writelines(raw_content)

pool = Pool(2)
pool.map(create_yaml_file, subset)
pool.close()
pool.join()

### Checking valid config and bbox

In [ ]:
def valid_config_bbox(yaml_file,data,index,yaml_folder='yamls'):
    yaml_files = os.listdir('yamls')
    if yaml_file in yaml_files:
        try:
            with open(os.path.join(yaml_folder, yaml_file), 'rb') as f:
                yaml_text = yaml.load(f)
                
            defa_source = yaml_text['sources']['default_source']
            if 'coverage' in defa_source:
                config=0
                bbox=0

            else:
                config=2
                bbox=1
            
            get_bbox=data[i]
            bbox_x0 = get_bbox[0]
            bbox_y0 = get_bbox[1]
            bbox_x1 = get_bbox[2]
            bbox_y1 = get_bbox[3]
            if bbox_x0=='None' or bbox_y0=='None' or bbox_x1=='None' or bbox_y1=='None':
                bbx=2
            else:
                bbox_x0 = float(bbox_x0)
                bbox_y0 = float(bbox_y0)
                bbox_x1 = float(bbox_x1)
                bbox_y1 = float(bbox_y1)
                if (abs(bbox_x0)>180.0) and (abs(bbox_x1)>180.0) and (abs(bbox_y0)>90) and (abs(bbox_y1)>90):
                    bbx=1
                else:
                    bbx=0
                    
            
        except yaml.YAMLError as exc:
            config,bbx= 1,1
            print (exc)
    else:
        config,bbx=1,1
        
    return config, bbx
        
  

In [ ]:

for i in range(4):
    print 'Proceso:' , i
    print subset[i]
    resul_valid, resul_bbox=valid_config_bbox('{0}.yml'.format(subset[i]),bbox_database,i)
    valid_config[i] = resul_valid
    valid_bbox[i] = resul_bbox
    


In [ ]:
#generate imgs if bbox is correct (None in bbox is -180,-90,180,90)
candidates_id = [idx for idx in range(len(valid_bbox)) if valid_bbox[idx]!=1 ]
print candidates_id


### Generate mapproxy image

In [ ]:
import io
import sys

from IPython.display import Image

from mapproxy.config.config import load_default_config, load_config
from mapproxy.config.loader import ProxyConfiguration
from mapproxy.wsgiapp import MapProxyApp

from multiprocessing import Pool

from six.moves.urllib_parse import unquote as url_unquote


def start_response(status, headers, exc_info=None):
    captured[:] = [status, headers, exc_info]
    return output.append


def environ_from_url(path):
    """From webob.request
    TOD: Add License.
    """
    scheme = 'http'
    netloc = 'localhost:80'
    if path and '?' in path:
        path_info, query_string = path.split('?', 1)
        path_info = url_unquote(path_info)
    else:
        path_info = url_unquote(path)
        query_string = ''
    env = {
        'REQUEST_METHOD': 'GET',
        'SCRIPT_NAME': '',
        'PATH_INFO': path_info or '',
        'QUERY_STRING': query_string,
        'SERVER_NAME': netloc.split(':')[0],
        'SERVER_PORT': netloc.split(':')[1],
        'HTTP_HOST': netloc,
        'SERVER_PROTOCOL': 'HTTP/1.0',
        'wsgi.version': (1, 0),
        'wsgi.url_scheme': scheme,
        'wsgi.input': io.BytesIO(),
        'wsgi.errors': sys.stderr,
        'wsgi.multithread': False,
        'wsgi.multiprocess': False,
        'wsgi.run_once': False,
    }
    return env


def get_path_info_params(yaml_text):
    sources = yaml_text['sources']['default_source']
    bbox_req = '-180,-90,180,90'
    if 'coverage' in sources:
        coverage = yaml_text['sources']['default_source']['coverage']
        bbox_req = ','.join([str(f) for f in coverage['bbox']])

    if 'layers'in yaml_text:
        lay_name = yaml_text['layers'][0]['name']

    return bbox_req, lay_name


def create_mapproxy_image(yaml_file):
    uuid = yaml_file.split('.')[0]
    print uuid
    with open(os.path.join(yaml_folder, yaml_file), 'rb') as f:
        yaml_text = yaml.load(f)

    captured = []
    output = []
    bbox_req, lay_name = get_path_info_params(yaml_text)
    
    path_info = ('/service?LAYERS={0}&FORMAT=image%2Fpng&SRS=EPSG%3A4326'
                 '&EXCEPTIONS=application%2Fvnd.ogc.se_inimage&TRANSPARENT=TRUE&SERVICE=WMS&VERSION=1.1.1&'
                 'REQUEST=GetMap&STYLES=&BBOX={1}&WIDTH=200&HEIGHT=150').format(lay_name, bbox_req)
    conf_options = load_default_config()
    # Merge both
    load_config(conf_options, config_dict=yaml_text)
    conf = ProxyConfiguration(conf_options, seed=False, renderd=False)
    
    
    # Create a MapProxy App
    app = MapProxyApp(conf.configured_services(), conf.base_config)
    # Get a response from MapProxyAppy as if it was running standalone.
    environ = environ_from_url(path_info)
    try:
        app_iter = app(environ, start_response)
        with open(os.path.join(images_folder, '{0}.png'.format(uuid)), 'wb') as img:
            img.write(app_iter.next())
        imag=0
        
    except: 
        imag=1 #error en la imagen
        
    return imag







In [ ]:
images_folder = 'images'
if not os.path.isdir(images_folder):
    os.mkdir(images_folder)

yaml_files=["{0}.yml".format(uuids[item]) for item in candidates_id]
for i in range (len(yaml_files)):
    idx= candidates_id[i]
    imag_exist[candidates_id[i]]= create_mapproxy_image(yaml_files[i])

    
#pool = Pool(4)
#pool.map(create_mapproxy_image, yaml_files)
#pool.close()
#pool.join()


### Saving CSV

In [ ]:
with open('checking_data.csv', 'wb') as f:
    writer = csv.writer(f,delimiter=' ')
    rows = zip(uuids,valid_config,valid_bbox,imag_exist)
    for row in rows:
        writer.writerow(row)